In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display as ipy_display
from pyvirtualdisplay import Display
import imageio
from IPython.display import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
torch.cuda.empty_cache()

In [ ]:
def preprocess(image):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 2D float array """
    image = image[35:195] # crop
    image = image[::2,::2,0] # downsample by factor of 2
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else (paddles, ball) just set to 1
    return np.reshape(image.astype(float).ravel(), [80,80])

In [ ]:
env = gym.make("Pong-v0")


In [ ]:
state, info = env.reset()

In [ ]:
screen = torch.tensor(preprocess(state)).unsqueeze(0).float()

In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(10, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 10 * 10, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.2)
        
        # Apply He initialization
        init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
        init.kaiming_normal_(self.conv2.weight, nonlinearity='relu')
        init.kaiming_normal_(self.conv3.weight, nonlinearity='relu')
        
        init.kaiming_normal_(self.fc1.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc2.weight, nonlinearity='relu')
        init.kaiming_normal_(self.fc3.weight, nonlinearity='relu')
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        
        x = x.view(-1, 128 * 10 * 10)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        
        x = self.dropout(F.relu(self.fc2(x)))
        
        x = self.fc3(x)
        
        return F.softmax(x, dim=1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

policy_net = PolicyNetwork().to(device)

In [ ]:
screen.shape

In [ ]:
window = torch.zeros((4, 80, 80))
window.shape

In [ ]:
window = torch.cat((screen, window), dim=0)

In [ ]:
output = policy_net(window.unsqueeze(0).to(device))

print("Output shape:", output.shape)
print("Output probabilities:", output)

In [ ]:
def get_action(policy_net, state, window, prior, method):
  if method == "random":
    action = np.random.choice(2) + 2
    probs = np.array([[0.5, 0.5]])
  if method == "gradient":
    screen = torch.tensor(preprocess(state)).unsqueeze(0).float()
    #screen = torch.where(screen != 0, torch.tensor(0.0), torch.tensor(1.0))
    screen = screen - prior
    screen = screen
    window = torch.cat((screen, window), dim=0)
    input_frame = window
    input_frame = input_frame.unsqueeze(0).to(device)
    probs = policy_net(input_frame)
    window = window[:9,:,:]
    prior = screen
    
    try:
        probs = probs.cpu().detach().numpy()
        action = np.random.choice(2, p=probs) + 2
    except ValueError:
        action = np.random.choice(2) + 2

  return action, probs, window, prior

In [ ]:
optimizer = optim.RMSprop(policy_net.parameters(), lr=0.001)

gamma = 0.99

policy_net

In [ ]:
longest_episode = 0
longest_episode_length = 0
longest_episode_path = ""
episode_lengths = []
loss_list = []
rolling_rewards = []
rolling_rewards_avg = []
rolling_time_avg = []

episode =  0
count = 0
cooldown = 0

In [ ]:
for episode in range(10000):
    
    state, info = env.reset()
    t = 0
    
    obs_history = []
    action_history = []
    episode_rewards = []
    saved_log_probs = []
    
    terminated, truncated = False, False
    
    window = torch.zeros((9, 80, 80))
    prior = torch.zeros((1, 80, 80))
    
    while not terminated:
        t+=1
        if (episode) % 25 == 0 | episode <= 25:
            action, probs, window, prior = get_action(policy_net, state, window, prior, "random")
        else:
            action, probs, window, prior = get_action(policy_net, state, window, prior, "gradient")
            
        observation, reward, terminated, truncated, info = env.step(action)
        
        obs_history.append(observation)
        episode_rewards.append(reward)
        action_history.append(action)
        saved_log_probs.append(1 + np.log(probs)[0][action-2])
        #print(torch.log(policy_net(torch.tensor(preprocess(state)).unsqueeze(0).float().to(device)))[0][action-2])
        
    #print(torch.log(policy_net(torch.tensor(preprocess(state)).unsqueeze(0).float().to(device)))[0][action-2])    
    #print(f"Episode {episode+1} finished after {t} time steps.")
    
    rolling_rewards.append(sum(episode_rewards))
    rolling_rewards_avg.append(np.mean(rolling_rewards[-100:]))
    discounted_rewards = []
    R = 0
    for i, r in enumerate(reversed(episode_rewards)):
        r = 10 * r + 1
        if episode_rewards[i] != 0: R = 0
        R = r + gamma * R
        discounted_rewards.insert(0, R)

    discounted_rewards = torch.tensor(discounted_rewards, requires_grad = True)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)

    #loss = -torch.mean(torch.mul(torch.stack(saved_log_probs), discounted_rewards.to(device)))
    loss = -torch.sum(discounted_rewards).to(device)
    loss_list.append(loss.cpu().detach().numpy())
    #print(f"Episode {episode + 1} loss: {loss}")
    episode_lengths.append(t)
    rolling_time_avg.append(np.mean(episode_lengths[-100:]))

    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(policy_net.parameters(), max_norm=1)
    optimizer.step()
    
    if (episode + 1) % 100 == 0:
        print("-"*50)
        print(f"Episode {episode+1} finished after {t} time steps.")
        print(f"Episode {episode+1} average time per 100: {np.mean(episode_lengths[-100:])} time steps.")
        print(f"Episode {episode + 1} loss: {loss}")
        print(f"Episode {episode + 1} discounted rewards: {torch.mean(discounted_rewards)}")
        print(f"Episode {episode + 1} saved log probs: {np.mean(saved_log_probs)}")
        print(f"Episode {episode + 1} Action Means: {np.mean(action_history)}")
        print(f"Episode {episode + 1} Reward Rolling Average per 100: {np.mean(rolling_rewards[-100:])}")
        print("-"*50)
    
env.close()
    

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(episode_lengths, marker='o')

plt.title('Episode Lengths Over Time')
plt.xlabel('Episode')
plt.ylabel('Length')

plt.show()
plt.savefig('results/pong_duration.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(loss_list, marker='o')

plt.title('Loss Over Time')
plt.xlabel('Episode')
plt.ylabel('Loss')

plt.show()
plt.savefig('results/pong_loss.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rolling_rewards_avg, marker='o')

plt.title('Avg Reward Over Time')
plt.xlabel('Episode')
plt.ylabel('Average Reward (per 100)')

plt.show()
plt.savefig('results/pong_avg_reward.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rolling_time_avg, marker='o')

plt.title('Avg Lifespan Over Time')
plt.xlabel('Episode')
plt.ylabel('Average Lifespan (per 100)')

plt.show()
plt.savefig('results/pong_avg_life.png', dpi=300, format='png', bbox_inches='tight')